# First/Second Return Contact Position

function to create json files for the return contact visual

In [1]:
import pandas as pd

In [38]:
pd.set_option('display.max_columns', None)

In [74]:
def generate_return_contact_jsons(df_shots, df_points, player):

    # only use matches with complete data
    df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])]

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', 'Detail', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    # only get UCLA player returns that are in
    # should we only get the results that are in or no?
    returns_ucla = combined[(combined['Player'] == player) & (combined['Shot'] == 2) & (combined['Result'] == 'In')]
    returns_ucla = returns_ucla[['Player', 'Hit (x)', 'Hit (y)', 'Point Winner', 'Stroke']].copy() # keep relevant columns

    # transforming shot contact data
    returns_ucla['shotContactX'] = returns_ucla['Hit (x)'] * 38.2764654418
    returns_ucla['shotContactY'] = (returns_ucla['Hit (y)'] - 11.8872) * 38.2764654418
    returns_ucla.drop(columns=['Hit (x)', 'Hit (y)'], inplace=True)
    # transform point winner
    returns_ucla['pointWonBy'] = returns_ucla['Point Winner'].apply(lambda x: player if x == 'host' else 'opponent') # dont need opponents name
    returns_ucla.drop(columns=['Point Winner'], inplace=True)
    # rename Stroke col and drop any non forehand backhand stroke
    returns_ucla.rename(columns={'Stroke': 'shotFhBh'}, inplace=True)
    returns_ucla = returns_ucla[returns_ucla['shotFhBh'].isin(['Forehand', 'Backhand'])]
    # rename Player column to shotHitBy
    returns_ucla.rename(columns={'Player': 'shotHitBy'}, inplace=True)

    # classify contact depth
    returns_ucla['shotContactY'] = returns_ucla['shotContactY'].apply(lambda y: -y if y > 0 else y)
    returns_ucla['depth'] = returns_ucla['shotContactY'].apply(
        lambda y: 'short' if y >= -455 else 'mid' if -455 > y > -490 else 'deep'
    )

    # save to json
    returns_ucla.to_json('ret_contact.json', orient='records')



    # creating distribution json

    distribution = returns_ucla.groupby('depth').apply(
        lambda df: pd.Series({
            'freq': len(df),
            'win_percentage': int((df['pointWonBy'] == df['shotHitBy']).mean() * 100)
        }),
        include_groups=False
    ).reset_index()

    # getting max and min win percentages
    max_win_percentage = distribution['win_percentage'].max()
    min_win_percentage = distribution['win_percentage'].min()
    distribution['maxMin'] = distribution['win_percentage'].apply(
        lambda x: 'max' if x == max_win_percentage else 'min' if x == min_win_percentage else 'no'
    )

    distribution['win_percentage'] = distribution['win_percentage'].astype(str) + '%'

    y_mapping = {
        'short': {'y': -420},
        'mid': {'y': -475},
        'deep': {'y': -515}
    }
    distribution['y'] = distribution['depth'].map(lambda d: y_mapping[d]['y'])

    # save to json
    distribution.to_json('ret_contact_dist.json', orient='records')

    return

In [ ]:
player = "Rudy Quan"
path = f"../../data/mens/{player}/combined.xlsx"
df_shots = pd.read_excel(path, sheet_name="Shots")
df_points = pd.read_excel(path, sheet_name="Points")

In [76]:
generate_return_contact_jsons(df_shots, df_points, player)